In [1]:
import json
import pandas as pd

In [2]:
with open('dpo_train.json','r') as f:
    train_data = json.load(f)
with open('dpo_eval.json','r') as f:
    eval_data = json.load(f)

with open('train_1.json','r') as f:
    train_1 = json.load(f)
with open('train_2.json','r') as f:
    train_2 = json.load(f)
with open('train_3.json','r') as f:
    train_3 = json.load(f)

complete_train_sum = {
    1:train_1,
    2:train_2,
    3:train_3
}


with open('eval_1.json','r') as f:
    eval_1 = json.load(f)
with open('eval_2.json','r') as f:
    eval_2 = json.load(f)
with open('eval_3.json','r') as f:
    eval_3 = json.load(f)

complete_eval_sum = {
    1:eval_1,
    2:eval_2,
    3:eval_3
}
train_df = pd.read_csv('train_bos.csv')
eval_df = pd.read_csv('eval_bos.csv')

In [3]:
selected_keys = []
for key in train_data:
    if len(train_data[key])>10:
        selected_keys.append(key)
for key in eval_data:
    if len(eval_data[key])>10:
        selected_keys.append(key)
len(selected_keys)

15

In [4]:
from rouge_score import rouge_scorer
import pandas as pd

# Sample inputs, reference summaries, and candidate summaries
data = [
    {
        "input_text": "The quick brown fox jumps over the lazy dog.",
        "reference_summary": "A quick fox jumps over a lazy dog.",
        "candidate_summaries": [
            "The quick fox leaped over the lazy dog.",
            "A brown fox jumps over a dog."
        ]
    },
    {
        "input_text": "Artificial intelligence is transforming industries.",
        "reference_summary": "AI is changing industries.",
        "candidate_summaries": [
            "AI is revolutionizing the world.",
            "Artificial intelligence is impacting all sectors."
        ]
    }
]

# Function to calculate ROUGE scores
def calculate_rouge(reference, candidates):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'])
    scores = []
    for candidate in candidates:
        score = scorer.score(reference, candidate)
        rouge1 = score['rouge1'].fmeasure
        rouge2 = score['rouge2'].fmeasure
        rougeL = score['rougeL'].fmeasure
        avg_score = (rouge1 + rouge2 + rougeL) / 3
        scores.append(avg_score)
    return scores

# Create the dataset
dataset = []
for item in data:
    input_text = item["input_text"]
    reference_summary = item["reference_summary"]
    candidate_summaries = item["candidate_summaries"]
    
    # Calculate ROUGE scores
    rouge_scores = calculate_rouge(reference_summary, candidate_summaries)
    
    # Determine the preferred candidate
    preferred_candidate = candidate_summaries[rouge_scores.index(max(rouge_scores))]
    
    # Add to dataset
    for i, candidate in enumerate(candidate_summaries):
        dataset.append({
            "Input Text": input_text,
            "Reference Summary": reference_summary,
            "Candidate Summary": candidate,
            "ROUGE Score": rouge_scores[i],
            "Preferred Candidate": preferred_candidate if candidate == preferred_candidate else None
        })

# Convert to DataFrame
df = pd.DataFrame(dataset)

# Save as CSV
# df.to_csv("preference_dataset.csv", index=False)
# print("Preference dataset created and saved as 'preference_dataset.csv'.")


In [5]:
dpo_dataset_train = {'ids':[],'prompt':[],
             'rejected':[],
             'chosen':[]}
dpo_dataset_eval = {'ids':[],
                    'prompt':[],
             'rejected':[],
             'chosen':[]}

for idx , row in train_df.iterrows():
    rank = train_data[row['video_ids']]
    if len(rank)<10:
        rank =[int(x) for x in rank.split(' ')]
        dpo_dataset_train['ids'].append(row['video_ids'])
        dpo_dataset_train['prompt'].append(row['transcripts'])
        dpo_dataset_train['rejected'].append(complete_train_sum[rank[2]][row['video_ids']])
        dpo_dataset_train['chosen'].append(complete_train_sum[rank[0]][row['video_ids']])
    else:
        dpo_dataset_train['ids'].append(row['video_ids'])
        dpo_dataset_train['prompt'].append(row['transcripts'])
        dpo_dataset_train['rejected'].append(complete_train_sum[1][row['video_ids']])
        dpo_dataset_train['chosen'].append(row['human_summaries'])
        

for idx, row in eval_df.iterrows():
    rank = eval_data[row['video_ids']]
    if len(rank) < 10:
        rank = [int(x) for x in rank.split(' ')]
        dpo_dataset_eval['ids'].append(row['video_ids'])
        dpo_dataset_eval['prompt'].append(row['transcripts'])
        dpo_dataset_eval['rejected'].append(complete_eval_sum[rank[2]][row['video_ids']])
        dpo_dataset_eval['chosen'].append(complete_eval_sum[rank[0]][row['video_ids']])
    else:
        dpo_dataset_eval['ids'].append(row['video_ids'])
        dpo_dataset_eval['prompt'].append(row['transcripts'])
        dpo_dataset_eval['rejected'].append(complete_eval_sum[1][row['video_ids']])
        dpo_dataset_eval['chosen'].append(row['human_summaries'])

# Save the train dataset
with open('dpo_dataset_train.json', 'w') as train_file:
    json.dump(dpo_dataset_train, train_file)

# Save the eval dataset
with open('dpo_dataset_eval.json', 'w') as eval_file:
    json.dump(dpo_dataset_eval, eval_file)

print("Train and eval datasets saved successfully!")

Train and eval datasets saved successfully!


In [6]:

curr_dpo_dataset_train = {'ids':[],'prompt':[],
             'rank 1':[],
             'rank 2':[],
             'rank 3':[],
            }
curr_dpo_dataset_eval = {
    'ids': [],
    'prompt': [],
    'rank 1': [],
    'rank 2': [],
    'rank 3': []
}
for idx , row in train_df.iterrows():
    rank = train_data[row['video_ids']]
    if len(rank)<10:
        rank =[int(x) for x in rank.split(' ')]
        curr_dpo_dataset_train['ids'].append(row['video_ids'])
        curr_dpo_dataset_train['prompt'].append(row['transcripts'])
        curr_dpo_dataset_train['rank 1'].append(complete_train_sum[rank[0]][row['video_ids']])
        curr_dpo_dataset_train['rank 2'].append(complete_train_sum[rank[1]][row['video_ids']])
        curr_dpo_dataset_train['rank 3'].append(complete_train_sum[rank[2]][row['video_ids']])

for idx, row in eval_df.iterrows():
    rank = eval_data[row['video_ids']]
    if len(rank) < 10:
        rank = [int(x) for x in rank.split(' ')]
        curr_dpo_dataset_eval['ids'].append(row['video_ids'])
        curr_dpo_dataset_eval['prompt'].append(row['transcripts'])
        curr_dpo_dataset_eval['rank 1'].append(complete_eval_sum[rank[0]][row['video_ids']])
        curr_dpo_dataset_eval['rank 2'].append(complete_eval_sum[rank[1]][row['video_ids']])
        curr_dpo_dataset_eval['rank 3'].append(complete_eval_sum[rank[2]][row['video_ids']])


# Save the train dataset
with open('curr_dpo_dataset_train.json', 'w') as train_file:
    json.dump(curr_dpo_dataset_train, train_file)

# Save the eval dataset
with open('curr_dpo_dataset_eval.json', 'w') as eval_file:
    json.dump(curr_dpo_dataset_eval, eval_file)

print("Train and eval datasets saved successfully in the updated format!")
    

Train and eval datasets saved successfully in the updated format!
